In [1]:
import numpy as np
import regex as re

In [84]:
vocab = []
encoded = {}

def simple_preprocessing(text):
    sample = text.strip().split()
    unique = []
    for i in sample: 
        if i not in unique: unique.append(i)
            
    vocab.extend(unique)
    onehot_encoder()

def onehot_encoder():
    for idx,word in enumerate(vocab):
        encoded[word] = np.zeros(len(vocab))
        encoded[word][idx] = 1
        
def io():
    x,y = [],[]
    for idx in range(len(vocab)-1):
        x.append(encoded[vocab[idx]])
        y.append(encoded[vocab[idx+1]])
    return np.array(x),np.array(y)

sample = "This is so good"    
simple_preprocessing(sample)
x,y = io()
#print(x,y)

In [85]:
class Word2Vec():
    def __init__(self):
        self.loss = []
        
    def fwd_prop(self,x):
        self.x1 = np.dot(self.w0.T,x) + self.b0
        self.x2 = np.dot(self.w1.T,self.x1) + self.b1
        self.x2 = np.exp(self.x2)
        self.x2 /= self.x2.sum()
        
    
    def back_prop(self,x):
        db1 = self.x2 * (1 - self.x2)
        dw1 = np.dot(self.x1,db1.T)
        
        b1, w1 = self.b1, self.w1
        
        self.b1 -= db1
        self.w1 -= dw1
        
        db0 = np.dot(w1,db1)
        dw0 = np.dot(x,db0.T)
        
        self.b0 -= db0
        self.w0 -= dw0
        
        #print(f"x1_shape = {self.x1.shape}, x2 shape = {self.x2.shape}, db1_shape = {db1.shape}, db1 = {db1}, dw1_shape = {dw1.shape}, dw1 = {dw1}db0_shape = {db0.shape}, db0 = {db0}, dw0_shape = {dw0.shape}, dw0 = {dw0}")

    
    def fit(self,x,y,hidden_units = 2, epochs=5,learning_rate=0.001):
        self.peripheral_units = x.shape[0]+1
        self.hidden_units = hidden_units
        
        self.w0 = np.random.normal(0,0.05,size = (self.peripheral_units, self.hidden_units))
        self.b0 = np.zeros((self.hidden_units,1))
        self.w1 = np.random.normal(0,0.05,size = (self.hidden_units, self.peripheral_units))
        self.b1 = np.zeros((self.peripheral_units,1))
        
        for iteration in range(epochs):
            for i in range(x.shape[0]):
                #print(f"Input = {x[i].shape}, Expected = {y[i].shape}, init_w0 = {self.w0.shape, self.w0}, init_b0 = {self.b0.shape, self.b0}, init_w1 = {self.w1.shape, self.w1}, init_b1 = {self.b1.shape, self.b1}")
                    
                self.fwd_prop(x[i].reshape(-1,1))
                l = self.x2 - y[i].reshape(-1,1)
                #print(f"loss = {l}, loss shape = {l.shape}")
                self.back_prop(x[i].reshape(-1,1))
                
                
    def predict(self,x):
        self.fwd_prop(x.reshape(-1,1))
        return self.x2

In [ ]:
file = open("datasets/The_Oxford_5000.txt","r")
data = file.read()
print(data[:100])
vocabulary = re.findall("\\n\w+",data)
vocabulary = [i[1:] for i in vocabulary]
vocabulary[:7]

In [108]:
word2vec = Word2Vec()
word2vec.fit(x,y,epochs=5)

In [109]:
word2vec.w0

array([[ 2.29719724e+00, -5.40598692e+00],
       [ 2.65603746e+00, -6.41708469e+00],
       [ 1.52307557e+00, -3.73524239e+00],
       [-5.66538214e-02, -6.09673620e-03]])

In [110]:
p = word2vec.predict(encoded["is"])

In [111]:
m = np.argmax(p)
p = np.zeros(p.shape)
p[m] = 1
p

array([[0.],
       [0.],
       [1.],
       [0.]])